# Question 1

In [2]:
import pandas as pd
import numpy as np

# Load submit_predictions.py
import submit_predictions

# import 'bank-full_2016.csv' into a DataFrame
test = pd.DataFrame.from_csv('fellows-test.csv', index_col=False)
train = pd.read_csv('fellows-train.csv', index_col=False)

# add the month, the day and the weekofyear
train['Date'] = pd.to_datetime(train['Date'])
train['Month'] = train.Date.dt.month
train['Day'] = train.Date.dt.day
train['WeekOfYear'] = train.Date.dt.weekofyear
train = train[train["Customers"] < 7300]
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
train['monthStr'] = train.Month.map(month2str)


test['Date'] = pd.to_datetime(test['Date'])
test['Month'] = test.Date.dt.month
test['Day'] = test.Date.dt.day
test['WeekOfYear'] = test.Date.dt.weekofyear
test['monthStr'] = test.Month.map(month2str)

# Attributes with missing values
print "NAN in Boston1: \n%s \n" % test.columns[pd.isnull(test).sum() > 0].tolist()

C:\Users\itc_admin\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


NAN in Boston1: 
[] 



In [3]:
print test.head()

   Store  WeekDay       Date  Open  Promo  StateHoliday  SchoolHoliday  Month  \
0      1        5 2015-07-31     1      1             0              1      7   
1      2        5 2015-07-31     1      1             0              1      7   
2      3        5 2015-07-31     1      1             0              1      7   
3      4        5 2015-07-31     1      1             0              1      7   
4      5        5 2015-07-31     1      1             0              1      7   

   Day  WeekOfYear monthStr  
0   31          31      Jul  
1   31          31      Jul  
2   31          31      Jul  
3   31          31      Jul  
4   31          31      Jul  


In [4]:
import sklearn 
from sklearn.ensemble.forest import RandomForestRegressor

# train data
# Explanatory variables
X_train  = train.drop(['Sales','StateHoliday','Date',"Customers",'monthStr'], 1)
# Target variable -> medv
Y_train = train['Sales']

from sklearn import linear_model

# linear regression
regr = RandomForestRegressor()
regr.fit(X_train, Y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [7]:
# Load the data
train_df = pd.read_csv('../input/train.csv', header=0)
test_df = pd.read_csv('../input/test.csv', header=0)

# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ['Pclass','Sex','Age','Fare','Parch']
nonnumeric_columns = ['Sex']

# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

# XGBoost doesn't (yet) handle categorical features automatically, so we need to change
# them to columns of integer values.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
# details and options
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['Survived']

# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = gbm.predict(test_X)

# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("submission.csv", index=False)

ImportError: No module named xgboost

In [5]:
# test data
# Explanatory variables
X_test  = test.drop(['StateHoliday','Date','monthStr'], 1)

# predict the value of the test set
A= list(regr.predict(X_test))

#send it
submit_predictions.submit(A, 'DATAGAGA')



0.20252413271218528